In [1]:
! pip install gensim
! pip install beautifulsoup4
! pip install wordcloud
! pip install tqdm

# или используем https://www.lfd.uci.edu/~gohlke/pythonlibs/#wordcloud

  Found existing installation: botocore 1.5.92
    Uninstalling botocore-1.5.92:
      Successfully uninstalled botocore-1.5.92
  Running setup.py bdist_wheel for wordcloud: started
  Running setup.py bdist_wheel for wordcloud: finished with status 'error'
  Complete output from command C:\Users\sanek\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\sanek\\AppData\\Local\\Temp\\pip-build-0sr37wnx\\wordcloud\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\sanek\AppData\Local\Temp\tmpdl1ep5popip-wheel- --python-tag cp35:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.5
  creating build\lib.win-amd64-3.5\wordcloud
  copying wordcloud\color_from_image.py -> build\lib.win-amd64-3.5\wordcloud
  copying wordcloud\tokenization.py -> build\lib.win-amd64-3.5\wordcloud
  copying wordcloud\wordclo

  Failed building wheel for wordcloud
Command "C:\Users\sanek\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\sanek\\AppData\\Local\\Temp\\pip-build-0sr37wnx\\wordcloud\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\sanek\AppData\Local\Temp\pip-y6f512cu-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\sanek\AppData\Local\Temp\pip-build-0sr37wnx\wordcloud\


Подгружаем данные:

In [1]:
import pandas as pd

train = pd.read_csv("labeledTrainData.tsv", header=0,
                    delimiter="\t", quoting=3)

In [2]:
train['review'].iloc[0]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

Видно, что в данных присутствуют элементы html.

- избавимся от элементов html  с помощью модуля BeautifulSoup: 
    http://nullege.com/codes/search/bs4.BeautifulSoup.getText
- оставим только буквы
- приведем все к нижнему регистру
- уберем стоп слова

In [3]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [ ]:
def review_to_words(raw_review):
    # your code is hear
    # 1. bs
    review_text = 
    # 2. regular expression
    letters_only = 
    # 3. to lower
    words = 
    # 4. drop stopwords
    words = 
    
    return words 

In [ ]:
review_to_words(train['review'].iloc[0])

Примените реализованную функцию к каждому предожению train

In [ ]:
from tqdm import tqdm

#num_reviews = train["review"].size 
num_reviews = 1000

clean_train_reviews = []

for i in tqdm(range(num_reviews)):
    clean_train_reviews.append(review_to_words(train["review"][i]))

Обучите word2vec

In [ ]:
num_features = 100    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

from gensim.models import word2vec

# подаем в обучение clean_train_reviews
model = word2vec.Word2Vec(clean_train_reviews, workers=num_workers, 
                          size=num_features, min_count = min_word_count, 
                          window = context, sample = downsampling)

model.init_sims(replace=True)

# модель можно сохранить
# model_name = ''
# model.save(model_name)

Теперь можно воспользjваться функциями поиска похожих / непохожих слов на данное
- model.wv.most_similar
- model.wv.doesnt_match
- etc

In [ ]:
# здесь можно посмотреть все слова, к которым мы имеем вектора
# model.wv.vocab

In [ ]:
model.wv.most_similar('mother')

In [ ]:
model.wv.doesnt_match("man woman child".split())

In [ ]:
model.wv.doesnt_match(["sun", "moon", "planet", "earth"])

In [ ]:
model.wv.most_similar("girl")

In [ ]:
model.wv.most_similar("horror")

Теперь выделите top 100 самых частотных слов и получите для них только что обученные вектора

In [ ]:
from nltk import FreqDist

fd = FreqDist()
# your code is hear

top_words = # your code is hear

In [ ]:
top_words_vec = # your code is hear

Визуализируем вектора. Для этого воспользуемся методом снижения размерности t-sne

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = # your code is hear

In [ ]:
print(top_words_vec.shape)
print(top_words_tsne.shape)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

x = []
y = []
for value in top_words_tsne:
    x.append(value[0])
    y.append(value[1])

plt.figure(figsize=(16, 16)) 
for i in range(100):
    plt.scatter(x[i],y[i])
    plt.annotate(top_words[i],
                 xy=(x[i], y[i]),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
plt.show()

Теперь нарисуйте облако слов

In [ ]:
from wordcloud import *
word_freq = [i for i in fd.most_common(100)]
wd = WordCloud(background_color = 'white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()